In [1]:
# Есть тестовый файл в формате .xslx. Его необходимо обработать и записать данные в базу.
# Необходимо:
#   - прочитать данные из файла
#   - обработать данные, привести к единому формату
#   - создать таблицу
#   - записать данные в таблцу

# Название колонок таблицы должно совпадать с названием колонок в файле.
# Поля содержащие внешние id лучше сделать строковыми для более удобной работы.
# Поля содержащие суммы должны иметь тип NUMERIC.

# Обязательные библиотеки для использования:
#   - pandas (для работы с данными)
#   - sqlalchemy (для работы с базой)

# Базу использовать локальную - sqlite.

In [2]:
import pandas as pd
from sqlalchemy import create_engine, ForeignKey, MetaData, Table, Column, String, Date, Numeric, Float
pd.options.mode.chained_assignment = None

In [3]:
engine = create_engine('sqlite:///test.db')

meta = MetaData()

# Создание всех необходимых таблиц
aggregation_method = Table(
    'aggregation_method', meta,
    Column('id', String, primary_key=True),
    Column('aggregation_method', String),
)

country = Table(
    'country', meta,
    Column('id', String, primary_key=True),
    Column('country', String),
)

city = Table(
    'city', meta,
    Column('id', String, primary_key=True),
    Column('city', String),
)

state = Table(
    'state', meta,
    Column('id', String, primary_key=True),
    Column('state', String),
)

geocode = Table(
    'geocode', meta,
    Column('id', String, primary_key=True),
    Column('ISO_3166_2', String),
)

traffic = Table(
    'traffic', meta,
    Column('id', String, primary_key=True),
    Column('aggregation_method_id', String, ForeignKey('aggregation_method.id')),
    Column('date', Date),
    Column('version', Float),
    Column('percent_of_baseline', Numeric),
    Column('centroid', String),
    Column('city_id', String, ForeignKey('city.id')),
    Column('state_id', String, ForeignKey('state.id')),
    Column('geocode_id', String, ForeignKey('geocode.id')),
    Column('country_id', String, ForeignKey('country.id')),
    Column('geography', String),
)

meta.create_all(engine)

In [4]:
# Чтение файла
df = pd.read_csv('covid_impact_on_airport_traffic.csv')

# Создание столбцов ID спомощью файкторизации
df['AggregationMethodId'] = (pd.factorize(df['AggregationMethod'])[0]+1).astype(str)
df['CountryId'] = (pd.factorize(df['Country'])[0]+1).astype(str)
df['CityId'] = (pd.factorize(df['City'])[0]+1).astype(str)
df['StateId'] = (pd.factorize(df['State'])[0]+1).astype(str)
df['GeocodeId'] = (pd.factorize(df['ISO_3166_2'])[0]+1).astype(str)

# Создание дата фрейма из CityId и City и удаление дубликатов
city_df = df[['CityId', 'City']].drop_duplicates()
# Переименование столбцов
city_df.rename(columns={'CityId': 'id', 'City': 'city'}, inplace=True)
# Запись в бд
city_df.to_sql(con=engine, index=False, name='city', if_exists='replace')

aggregation_method_df = df[['AggregationMethodId', 'AggregationMethod']].drop_duplicates()
aggregation_method_df.rename(columns={'AggregationMethodId': 'id', 'AggregationMethod': 'aggregation_method'},
                             inplace=True)
aggregation_method_df.to_sql(con=engine, index=False, name='aggregation_method', if_exists='replace')

country_df = df[['CountryId', 'Country']].drop_duplicates()
country_df.rename(columns={'CountryId': 'id', 'Country': 'country'}, inplace=True)
country_df.to_sql(con=engine, index=False, name='country', if_exists='replace')

state_df = df[['StateId', 'State']].drop_duplicates()
state_df.rename(columns={'StateId': 'id', 'State': 'state'}, inplace=True)
state_df.to_sql(con=engine, index=False, name='state', if_exists='replace')


geocode_df = df[['GeocodeId', 'ISO_3166_2']].drop_duplicates()
geocode_df.rename(columns={'GeocodeId': 'id'}, inplace=True)
geocode_df.to_sql(con=engine, index=False, name='geocode', if_exists='replace')

traffic_df = df[[
    'AggregationMethodId',
    'Date',
    'Version',
    'PercentOfBaseline',
    'Centroid',
    'CityId',
    'StateId',
    'GeocodeId',
    'CountryId',
    'Geography']]
traffic_df.rename(columns={
    'AggregationMethodId': 'aggregation_method_id',
    'Date': 'date',
    'Version': 'version',
    'PercentOfBaseline': 'percent_of_baseline',
    'Centroid': 'centroid',
    'CityId': 'city_id',
    'StateId': 'state_id',
    'GeocodeId': 'geocode_id',
    'CountryId': 'country_id',
    'Geography': 'geography'
}, inplace=True)
traffic_df.to_sql(con=engine, index_label='id', name='traffic', if_exists='replace')

In [5]:
query = '''
SELECT city.city, traffic.geography
FROM traffic INNER JOIN city
ON city.id=traffic.city_id
WHERE traffic.percent_of_baseline == 99
'''

print(pd.read_sql(query, engine))

              city                                          geography
0          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
1          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
2          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
3          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
4          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
5          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
6          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
7          Calgary  POLYGON((-113.981866836548 51.1392131913567, -...
8     Leduc County  POLYGON((-113.568120002747 53.3110445425655, -...
9         Richmond  POLYGON((-123.136525154114 49.1980971490671, -...
10        Winnipeg  POLYGON((-97.2094345092773 49.8979517850609, -...
11        Winnipeg  POLYGON((-97.2094345092773 49.8979517850609, -...
12        Winnipeg  POLYGON((-97.2094345092773 49.8979517850609, -...
13        Winnipeg  